# Abstract Interpretation of SPARK Programs

Traditionally, SPARK programs have been analysed by the Examiner using data flow and information flow analysis.  This study does not consider the proof features of the Examiner (VCG generator) but rather investigates whether the same or better results to flow analysis can be achieved using abstract interpretation. Better results may be achievable using abstract interpretation as it might be possible to determine some non-executable paths and exclude them from the analysis.  To achieve this goal, two of the more advanced techniques of abstract interpretation - variable analysis and path analysis are needed. 

The SPARK language was developed to achieve accurate and straightforward flow analysis. Many of the restrictions placed on Ada by SPARK to make it amenable to flow analysis may also be advantageous for abstract interpretation.  


## Abstract Interpretation
Abstract interpretation is commonly viewed as having three stages:
1. Translate
2. Merge
3. Widen

These three stages are applied to each statement of the given source text until it has been completed.

Translate converts a statement into a simple model representing the statement.  Merge takes the model of the statement and the abstractions of the immediately preceding statements (in general, there may be more than one due to gotos if statements and loops) and merges them into a single abstraction for the statement.  The abstraction is an approximation of the state at the statement.  Generally, it consists of an entry for each variable and an approximation of the range of values that variable may have at the statement.  The Widen stage is typically used after loops to widen the approximate range of possible values a variable may have to represent executing the loop multiple times.

What is interesting about these stages is that they can be adapted to suit a number of different analyses but still fit within the framework of calling each of the three stages for each statement.

For instance,  constant analysis is may be used to obtain an approximation of the range of values a variable may have at a particular statement based on the value of constants within the source text.  Variable analysis is similar but more complex based on the expressions assigned to variables within the source text.

To perform abstract interpretation, an abstract model of the source text needs to be constructed consisting of a model of each statement (the translation) and a sequence of abstractions representing each of the previous statements.  Merge consolidates the immediately preceding statements with the model of the current statement to obtain an abstraction for the current statement which is appended to the sequence.  Merging is a simple operation when the statement has only one immediate predecessor but becomes a little more complex around if statements, loops and the targets of goto statements.

SPARK has the advantage over general programming language in that, other than in a loop, an immediately preceding statement cannot be later in text than the current statement, simplifying the sequence of preceding statements that need to be maintained.  As SPARK is modular and each subprogram is essentially self-contained and only variables used within the subprogram need to be in the abstractions, very little extra context has to be maintained and the sequence of abstractions can be discarded after completing the analysis of the subprogram.

This study starts with the simple but very important check for SPARK variable defindness.   In Ada terms, ideally is every variable initialised to a valid value before it is read.  

## Abstract Interpretation of Defindness
Abstract interpretaion uses an abstraction based on an approximation the range of values a variable may have at a given statement.  In variable defindness we are concerned whether a variable has been assigned a (preferably valid) value but are not concerned with the precise value.  A representatation of the values of defindness needs to used.

Consider a model to represent the values of defindness. Each variable may have a few possible values:

1. Uninitialized
2. Unsound
3. Assigned
4. Read

Uninitalized, the variable has not been assigned a value, Unsound, the variable has been assigned an Unitialized or Unsound value, Assigned the variable has been assigned a value not dependent on an uninitialized or unsound value and Read, the variable has been read. The value Unsound represents a value that has been derived from an Unintialized value.  An Assigned value is not necessarily a Valid value in the Ada sense as a value could be Assigned to a variable which is out of range and therefore an invalid value.  A more sophisticated abstract interpretation technique may be able to approximate the range of values a variable may have.

The statements of a subprogram Read or Assign variables. The Read of a variable with an Uninitialized value is erroneous and the Read of a variable with an Unsound value is of questionable Validity.  Two successive Assignments to the same variable without an intervening Read of the variable means (in SPARK) the value of the first assignment is unused and therefore may suggest a programming mistake. 

In a single statement a variable may be both read and assigned but in SPARK expressions do not have side-effects so the all the variables on the right hand side of an assignment statement are only Read.  On the left hand side only array indices are Read although, unusually in object dclarations, more than one variable may be assigned.  In a subprogram call, formal parameters of mode **in** or **in out** are modelled as Reads of the corresponding actual parameters.  Similarly, formal paramaters of mode **out** or **in out** are modelled as Assignment to the corresponding actual actual parameters in a procedure call statement.  A subprogram can have many parameters and globals leading to the model of a statement potentially having multiple Reads of variables (possibly of the same variable) and multiple Assignments of variables.  All assignments in a single statement are to unique variables.

The proposed translation first considers all of variables read by the statement and then those that are assigned by the statement. This avoids ambiguity when the same variable is both read and written by the statement.  Each read and assignment of a variable will have a separate entry in the translation.  For instance:

    X := X + X + Y;

would be translated as:

    Read_Var   (X)
    Read_Var   (X)
    Read_Var   (Y)
    Assign_Var (X)

To keep an association between the translation and the statement a position and a translation item is needed.  For simplicity, in this study, only one statement per line is assumed and so the can just be the line number.

Assuming the above statement is on line 10, the translation becomes:

    Read   (X, (10, 1))
    Read_  (X, (10, 2))
    Read   (Y, (10, 3))
    Assign (X, (10, 4))

It may not be necessary to record the two Reads of X, one may be sufficient as a Read does not change the state of the variable.

The call of a procedure, 

**procedure** P (A : **in** Integer; B : **in out** Integer; C : **out** Integer);

on line 20

    P (X, Y, X);

would be translated as (assuming no dependency relation given):

    Read_Var   (X, (20, 1))
    Read_Var   (Y, (20, 2))
    Assign_Var (Y, (20, 3))
    Read_Var   (X, (20, 4))
    Read_Var   (Y, (20, 5))
    Assign_Var (X, (20, 6))

It is important that the Reads of variables use the value of the variable from the merged value of the immidiately preceding statements rather than the values Assigned in the by the procedure call.

Each statement has an abstraction representing the state of all variables on completion of execution of the statement.  The abstraction at the statement is formed by the merging the abstractions of all immediately preceding statements and applying the effects of the current statement.  Commonly, an abstraction has a notion of the range of values that a variable may have at the statement.  For defindness the maximum range is [Uninitalized, Assigned], athough in defindness there is not necessarily an order to these posible values but for simplicity an oreder is assumed Unintialized < Unsound < Assigned < Read.  A merge has to take the abstractions from the immediately preceding statements and the translation of the statement form this pair of values.

First consider the merge of the immediately preceding statements.  This part of the merge determines the widest range of values that the variable could take from the abstractions of all of its immediately preceding statements.  That is, the lower bound is taken to be lowest bound of all its immediate predecessors and the high bound from the highest.

Next consider the variables that are Read by the statement.  SPARK flow analysis reports on all uses of an uninialized variable, whether this be unconditional (it will always happen when executing the program) or conditional (it will happen dependent on the excution path taken when the program is run). The abstract interpretation model of defindness needs to do this too.

If a variable Read by a statement has a range in the merged immediately prceeding statements of [Uninitialized, Uninitialized], then this is clearly an unconditional Read of an Uninitialized variable and should be reported.  In this case the range remains as [Uninitialized, Uninitialized] as a Read of the variable does not change its value - it is still Uninitialized.  If the lower bound of the variable in the merged statements is Uninitialized but the upper bound is Assigned or Read, then this is the conditional use of an uninitialized variable and should be reported as such.  The lower bound should remain as Uninitialized as the Read of the variable has not changed this, however if the upper bound is Assigned or Read it should become Read as the previously Assigned or Read variable has been Read.

Determining the range of a Unsound variable is similar.  If the range of the variable is [Uninitialized, Unsound] or [Unsound, Unsound] then a Read of the variable does not change its value.  If the range of the variable is [Unsound, Assigned] then it becomes [Unsound, Read] after the Read of the variable and a variable with the range [Unsound, Read] remains unchanged. 

In summary here are the rules for the effects of a Read of a variable on the variable:

    Merged Bounds                   <Read_Var  New Bounds   
    [Uninitialized, Uninitialized]     ->     [Uninitialized, Uninitialized] (1)
    [Uninitialized, Unsound]           ->     [Uninitialized, Unsound] 
    [Uninitialized, Assigned]          ->     [Uninitialized, Read]          (2)
    [Unsound, Unsound]                 ->     [Unsound, Unsound]
    [Unsound, Assigned]                ->     [Unsound, Read]
    [Unsound, Read]                    ->     [Unsound, Read]
    [Assigned, Assigned]               ->     [Assigned, Read]
    [Assigned, Read]                   ->     [Assigned, Read]
    [Read, Read]                       ->     [Read, Read]

Notes:
(1) Issue an unconditional Read of an uninitialized variable message.
(2) Issue a conditional Read of an uninitialized variable message.

It is not clear that a message should be issued yet regarding the Read of an Unsound variable.

Assigning to a variable potentially changes its value dependent on any variables Read in determining its (defindness) value.  SPARK flow analysis also reports on two successive assignments to a variable without an intervining read of the variable.   The value of the first assignment is not used and may be a mistake in the program.  Such assignments should be detected with abstract interpretation too.

First consider bounds of the Read variables in this statement, after the Read variable action has been applied as described above, that are used in determing the defindness value of the assignment.  If any of bounds of the variables Read have a lower bound of less than Assigned, the the value is questionable and the lower bound of the assignment value should set to Unsound.  If the upper bound is Uninitialized or Unsound, then the upper bound should also be set to Unsound as the (actual as opposed to defindness) value assigned must have an Unsound defindness value.

In summary the effects of the variables Read in determiming the value of the assignment are below but only the possible bounds as given above are included:

    Bounds of Read Variable            Bounds of Assignment Value
    [Uninitialized, Uninitialized] -> [Unsound, Unsound]   
    [Uninitialized, Unsound]       -> [Unsound, Unsound]   
    [Uninitialized, Read]          -> [Unsound, Assigned] 
    [Unsound, Unsound]             -> [Unsound, Unsound]
    [Unsound, Read]                -> [Unsound, Assigned]
    [Assigned, Read]               -> [Assigned, Assigned]
    [Read, Read]                   -> [Assigned, Assigned]

There is one further check that is needed to detect successive assignments without an intervening read.  If the Assigned variable is not one of the Read variables used in determining the assignment value and its bounds obtained from the merged bounds of the immediately preceding statements is [Assigned, Assigned], then its value has not been read since its last assignment and a warning message should be issued.

It would be possible to check that read only variables (constants in Ada or **in** mode parameters) are not Assigned but Ada compilers already do this check.


So far individual statements have been considered but the basic analysis unit in SPARK is the subprogram and further actions than just Read variable and Assign variable are needed as well as more analysis of the final abstract state of the subprogram.

The subprogram declaration has to be translated into a model.  A subprogram may have mode **in**, **out**, or **in out** parameters. To keep the astraction simple, the mode **in** and **in out** parameters are modelled as a pre assigned input auxillary variable (assigned when the subprogram is called and have bounds [Assigned, Assigmed]) and the mode **out** and **in out** parameters are modelled as output auxilliary variables which are Assigned on completion of the subprogram (the **end** keyword) and initially have bounds [Uninitialized, Uninitialized]. 

The the formal parameters are treated as variables local to the subprogram.  A formal parameter which has mode **in** or **in out** is assigned from the input auxillary variable giving it bounds [Assigned, Read].  On completion of the subprogram interpretation formal parameters of mode **in out** or **out** are assigned to the output auxilliary variables.  For example, the procedure declaration:

    procedure P (A : in Integer; B : in out Integer; C : out Integer);

will be translated as:

    In_Param   (A__in,  (1, 1))      (A__in,  (Assigned, (1, 1)), (Assigned, (1 ,1)))
    Declare    (A, (1, 2))           (A,      (Uninitialized (1, 2)), (Uninintialized, 91, 2)))
    Read_Var   (A__in, (1, 3))       (A__in,  (Assigned, (1, 1)), (Read, (1, 3)))
    Assign_Var (A,     (1, 4))       (A,      (Assigned, (1, 4)), (Assigned, (1, 4)))
    In_Param   (B__in,  (1, 5))      (B__in,  (Assigned, (1, 5)), (Assigned, (1, 5)))
    Out_Param  (B__out, (1, 6))      (B__out, (Uninitialized, (1, 6)), (Uninitialized, (1, 6)))
    Declare    (B,      (1, 7))      (B,      (Uninitialized, (1, 7)), (Unintialized, (1, 7)))
    Read_Var   (B__in,  (1, 8))      (B__in,  (Assigned, (1, 5)), (Read, (1, 8)))
    Assign_Var (B,      (1, 9))      (B,      (Assigned, (1, 9)), (Assigned, (1, 9)))
    Out_Param  (C__out, (1, 10))     (C__out, (Uninitialized, (1, 10)), (Uninitialized, (1, 10)))
    Declare    (C,      (1, 11))     (C,      (Uninitialized, (1, 11)), (Uninitialized, (1, 11)))

                                 (A__in,  (Assigned, (1, 1)), (Read, (1, 3)))
                                 (B__in,  (Assigned, (1, 4)), (Read, (1, 8)))
                                   (B__out, (Uninitialized, (1, 6)), (Uninitialized, (1, 6)))
                                   (C__out, (Uninitialized, (1, 10)), (Uninitialized, (1, 10)))
                                   (A,      (Assigned, (1, 4)), (Assigned, (1, 4)))
                                   (B,      (Assigned, (1, 9)), (Assigned, (1, 9)))
                                   (C,      (Uninitialized, (1, 11)), (Uninitialized, (1, 11)))

In SPARK all parameters of a function subprogram must be mode **in** so, in principle, the input auxillary variables can dispensed with and the formal parameters used directly in the interpretation but they would have initial bounds of [Assigned, Assigned].  The return value modelled as an auxillary output variable that is Assigned to by the return statement.

SPARK requires all global variables used by a subprogram to be included in the subprogram declaration these also have a mode **in**, **in out** or **out** and are treated simlarly to formal parameters.  However, Ada compilers do not check the correct use (according to their mode) of global variables as is done with parameters.  For instance, an Ada compiler does not report the assignment to a mode **in** global variable.  Extra checks for correct use of global variables are needed when interpreting assignments.

Local variable declarations also have to considered as these introduce new variables into the state abtraction.  Local variables will have initial bounds of [Uninitialized, Uninitialized] but if they have an initialization expression it will be translated as an Assignment.


Some rules are needed for merge, for the moment consider only simple source text which has no if, or loop statements (straight line code):

    Previous Value        Statement Model  New Value   
    Uninitialized <merge> Read_Var         -> Unsound  --  Use of an uninitialised variable
    Uninitialized <merge> Assign_Var       -> Assigned
    Read          <merge> Read_Var         -> Read
    Read          <merge> Assign_Var       -> Assigned
    Assigned      <merge> Read_Var         -> Read
    Assigned      <merge> Assign_Var       -> Assigned --  Previously assigned value is unused
    Unsound       <merge> Read_Var         -> Unsound  --  The value read may be unsound
    Unsound       <merge> AssignVar        -> Assigned

In SPARK there is not a statement to Uninitialise a variable or to set it as unsound so these states do not appear in the Statement Model above.  A variable is uninitialised when declared, so the model of a declaration is Uninitialized but it has no previous state.  The Unsound state can only be entered by a Read of an Uninitialised state or the Read of an Unsound state. Consequently, if a Previous State is Read then it must have previously been Assigned.  

As the abstraction has a pair of states to represent the "range" of values a variable might have at a paricular statement, the Previous State for the variable forms one of the pair and the newly merged state is the other.

When There is more than one immediately preceding statement (in SPARK after an if, case or loop statement) these rules will have to be applied for each predecessor and then the state pairs could show a significant difference depending on the path taken eg., (Assigned, Uninitialised) meaning that on one path the variable associated with the state pair is Uninitialised. 

### Straight Line Code

First, consider the following simple SPARK procedure taken from early SWES courses.

    procedure Swap (X : in out Integer; Y : in out Integer) is
      Temp : Integer;
    begin
      Temp := X;
      X := Y;
      Y := Temp;
    end Swap;

As it is written there are no uses of uninitialised variables.

Using the ideas for models and abstractions the following spreadsheet was constructed to demonstrate using abstract interpretation to check for defindness.

![image.png](attachment:84ecc425-647b-4f3d-ac88-001f7b003066.png)

#### Python Version of Spreadsheet
The Translate operation of the Abstract Interpretation is done by hand.

An enumeration is introduced to represent the possible states of a variable and two named tuples are declared to  of an action and the action and the variable to which it refers.

Two sequences are now declared, one to record where each statement starts and one containing the expanded and translated statements.

When conditional statements are considered the structure will be more complex but first consider straight line code in which only the current statement translation and the abstraction current at the immediatly preceding statement have to be merged.
A set of rules for merging and a data structure representing the abstract state is required.  The merge rules may be represented as a matrix and the abstract state, looking at the spreadsheet could be an extension of the Expansion structure.

In [103]:
from enum import Enum
class Defindness (Enum):
    UNINITIALIZED = 1
    READ          = 2
    ASSIGNED      = 3
    UNSOUND       = 4
    @classmethod
    def pos (cls, state):
        return state.value - 1

class Translate_Actions (Enum):
    READ_VAR    = 1
    ASSIGN_VAR  = 2
    IN_PARAM    = 3
    OUT_PARAM   = 4
    DECLARE_VAR = 5
    @classmethod
    def pos (cls, act):
        return act.value - 1
    
State  = Enum('State',  ['UNINITIALIZED', 'READ', 'ASSIGNED', 'UNSOUND'])
Action = Enum('Action', ['READ_VAR', 'ASSIGN_VAR', 'IN_PARAM', 'OUT_PARAM', 'DECLARE_VAR'])

import collections
Position  = collections.namedtuple ('Postition', ['line', 'expand'])
Expansion = collections.namedtuple ('Expansion',['var', 'action', 'position'])

translation = [
    [Expansion("X__in",  Action.IN_PARAM,    Position(0, 1)),  # Statement 0 represents declaring a 
     Expansion("X__out", Action.OUT_PARAM,   Position(0, 2)),  # variable for each in and out parameter.
     Expansion("Y__in",  Action.IN_PARAM,    Position(0, 3)),  # Variables with the parameter names
     Expansion("Y__out", Action.OUT_PARAM,   Position(0, 4)),  # are declared for use within the
     Expansion("X",      Action.DECLARE_VAR, Position(0, 5)),  # translation of the body of the
     Expansion("Y",      Action.DECLARE_VAR, Position(0, 6))], # subprogram.
     
    [Expansion("X__in",  Action.READ_VAR,     Position(1, 1)),  # The in parameters are assigned to the
     Expansion("Y__in",  Action.READ_VAR,     Position(1, 2)),  # local variables X and Y.
     Expansion("X",      Action.ASSIGN_VAR,   Position(1, 3)),
     Expansion("Y",      Action.ASSIGN_VAR,   Position(1, 4))],

    [Expansion("Temp",   Action.DECLARE_VAR,  Position(2, 1))], # The declaration of Temp.
                                                                # The begin keyword is not translated?
    [Expansion("X",      Action.READ_VAR,     Position(4, 1)),
     Expansion("Temp",   Action.ASSIGN_VAR,   Position(4, 2))],

    [Expansion("Y",      Action.READ_VAR,     Position(5, 1)),
     Expansion("X",      Action.ASSIGN_VAR,   Position(5, 2))],

    [Expansion("Temp",   Action.READ_VAR,     Position(6, 1)),
     Expansion("Y",      Action.ASSIGN_VAR,   Position(6, 2))],

    [Expansion("X",      Action.READ_VAR,     Position(7, 1)),  # end keyword denotes updating the out
     Expansion("Y",      Action.READ_VAR,     Position(7, 2)),  # parameters.
     Expansion("X__out", Action.ASSIGN_VAR,   Position(7, 3)),
     Expansion("Y__out", Action.ASSIGN_VAR,   Position(7, 4))]]

In [104]:
translation

[[Expansion(var='X__in', action=<Action.IN_PARAM: 3>, position=Postition(line=0, expand=1)),
  Expansion(var='X__out', action=<Action.OUT_PARAM: 4>, position=Postition(line=0, expand=2)),
  Expansion(var='Y__in', action=<Action.IN_PARAM: 3>, position=Postition(line=0, expand=3)),
  Expansion(var='Y__out', action=<Action.OUT_PARAM: 4>, position=Postition(line=0, expand=4)),
  Expansion(var='X', action=<Action.DECLARE_VAR: 5>, position=Postition(line=0, expand=5)),
  Expansion(var='Y', action=<Action.DECLARE_VAR: 5>, position=Postition(line=0, expand=6))],
 [Expansion(var='X__in', action=<Action.READ_VAR: 1>, position=Postition(line=1, expand=1)),
  Expansion(var='Y__in', action=<Action.READ_VAR: 1>, position=Postition(line=1, expand=2)),
  Expansion(var='X', action=<Action.ASSIGN_VAR: 2>, position=Postition(line=1, expand=3)),
  Expansion(var='Y', action=<Action.ASSIGN_VAR: 2>, position=Postition(line=1, expand=4))],
 [Expansion(var='Temp', action=<Action.DECLARE_VAR: 5>, position=Posti

In [105]:
statements = [
    0,  # Represent setting values of in parameters in a procedure call.
    7,  # procedure Swap (X : in out Integer; Y : in out Integer) is
    11, # Temp : Integer;
    12,  # Temp := X;
    14, # X := Y;
    16, # Y := Temp;
    18  # end; Represent setting values of out parameters.
]

In [106]:
statements

[0, 7, 11, 12, 14, 16, 18]

When conditional statements are considered the structure will be more complex but first consider straight line code in which only the current statement translation and the abstraction current at the immediatly preceding statement have to be merged.
A set of rules for merging and a data structure representing the abstract state is required.  The merge rules may be represented as a matrix and the abstract state, looking at the spreadsheet could be an extension of the Expansion structure.


|                  |READ_VAR     |ASSIGN_VAR |IN_PARAM |OUT_PARAM     |DECLARE_VAR   |
|------------------|-------------|-----------|---------|--------------|--------------|
|**UNINITIALIZED** |UNSOUND      |ASSIGNED   |ASSIGNED |UNINITIALIZED |UNINITIALIZED |
|**READ**          |READ         |ASSIGNED   |ASSIGNED |UNINITIALIZED |UNINITIALIZED | 
|**ASSIGNED**      |READ         |ASSIGNED   |ASSIGNED |UNINITIALIZED |UNINITIALIZED | 
|**UNSOUND**       |UNSOUND      |ASSIGNED   |ASSIGNED |UNINITIALIZED |UNINITIALIZED |

There is a lot of redundancy in this table, it is immaterial what the current state is for the actions IN_PARAM, OUT_PARAM or DECLARE_VAR.  Similarly for ASSIGN_VAR except that two successive assigns without an intervening read is a reportable state change.  The matrix is only small so the redundancy is of little concern. 

In Python the matrix is represented as a list of lists.

In [135]:
  # READ_VAR      ASSIGN_VAR     IN_PARAM        OUT_PARAM            DECALRE_VAR
merge_matrix = [
  # READ_VAR      ASSIGN_VAR     IN_PARAM        OUT_PARAM            DECALRE_VAR
 [State.UNSOUND, State.ASSIGNED, State.ASSIGNED, State.UNINITIALIZED, State.UNINITIALIZED],# UNINITIALIZED
 [State.READ,    State.ASSIGNED, State.ASSIGNED, State.UNINITIALIZED, State.UNINITIALIZED],# READ
 [State.READ,    State.ASSIGNED, State.ASSIGNED, State.UNINITIALIZED, State.UNINITIALIZED],# ASSIGNED
 [State.UNSOUND, State.ASSIGNED, State.ASSIGNED, State.UNINITIALIZED, State.UNINITIALIZED] # UNSOUND
]

In [134]:
merge_matrix [Defindness.pos(State.UNSOUND)][Translate_Actions.pos(Action.READ_VAR)]

<State.UNSOUND: 4>

The detection of errant actions of reading an uninitialized variable or an unsound value is directly available from the matrix but two assignments of the same variable without an intervening read is not.  Nor is assignment of an unsound value.  Is Unread another state? - but then what happens to assigned?

Consider the structure of the abstract state.  It contains an element for each variable in the subprogram.  Each element contains the merged state of the variable after the current instruction.  For error reporting the position may be needed too.  The requirements look the same as for the Expansion. 

In [ ]:
State = Enum('Action', ['UNINITIALIZED', 'READ', 'ASSIGNED', 'UNSOUND'])

Var_State = collections.namedtuple ('Var_State',['var', 'state', 'position'])
